# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
region_1 = pd.read_csv('/datasets/geo_data_0.csv')
region_2 = pd.read_csv('/datasets/geo_data_1.csv')
region_3 = pd.read_csv('/datasets/geo_data_2.csv')

### Регион 1.

In [3]:
region_1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
region_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
region_1.duplicated().sum()

0

In [6]:
region_1['id'].duplicated().sum()

10

In [7]:
region_1[region_1.duplicated(subset=['id'], keep=False)].sort_values(by='id')

,id,f0,f1,f2,product
66136,74z30,1.084962,-0.312358,6.990771,127.643327
64022,74z30,0.741456,0.459229,5.153109,140.771492
51970,A5aEY,-0.180335,0.935548,-2.094773,33.020205
3389,A5aEY,-0.039949,0.156872,0.209861,89.249364
69163,AGS9W,-0.933795,0.116194,-3.655896,19.230453
42529,AGS9W,1.454747,-0.479651,0.683380,126.370504
931,HZww2,0.755284,0.368511,1.863211,30.681774
7530,HZww2,1.061194,-0.373969,10.430210,158.828695
63593,QcMuo,0.635635,-0.473422,0.862670,64.578675
1949,QcMuo,0.506563,-0.323775,-2.215583,75.496502


In [8]:
region_1 = region_1.drop_duplicates(subset=['id'], keep='last')

In [9]:
region_1['id'].duplicated().sum()

0

### Регион 2.

In [10]:
region_2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [11]:
region_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [12]:
region_2.duplicated().sum()

0

In [13]:
region_2['id'].duplicated().sum()

4

In [14]:
region_2[region_2.duplicated(subset=['id'], keep=False)].sort_values(by='id')

,id,f0,f1,f2,product
5849,5ltQ6,-3.435401,-12.296043,1.999796,57.085625
84461,5ltQ6,18.213839,2.191999,3.993869,107.813044
1305,LHZR0,11.170835,-1.945066,3.002872,80.859783
41906,LHZR0,-8.989672,-4.286607,2.009139,57.085625
2721,bfPNe,-9.494442,-5.463692,4.006042,110.992147
82178,bfPNe,-6.202799,-4.820045,2.995107,84.038886
47591,wt4Uk,-9.091098,-8.109279,-0.002314,3.179103
82873,wt4Uk,10.259972,-9.376355,4.994297,134.766305


In [15]:
region_2 = region_2.drop_duplicates(subset=['id'], keep='last')

In [16]:
region_2['id'].duplicated().sum()

0

### Регион 3.

In [17]:
region_3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [18]:
region_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [19]:
region_3.duplicated().sum()

0

In [20]:
region_3['id'].duplicated().sum()

4

In [21]:
region_3[region_3.duplicated(subset=['id'], keep=False)].sort_values(by='id')

,id,f0,f1,f2,product
45404,KUPhW,0.231846,-1.698941,4.990775,11.716299
55967,KUPhW,1.211150,3.176408,5.543540,132.831802
11449,VF7Jo,2.122656,-0.858275,5.746001,181.716817
49564,VF7Jo,-0.883115,0.560537,0.723601,136.233420
44378,Vcm5J,-1.229484,-2.439204,1.222909,137.968290
95090,Vcm5J,2.587702,1.986875,2.482245,92.327572
28039,xCHr8,1.633027,0.368135,-2.378367,6.120525
43233,xCHr8,-0.847066,2.101796,5.597130,184.388641


In [22]:
region_3 = region_3.drop_duplicates(subset=['id'], keep='last')

In [23]:
region_3.duplicated().sum()

0

### Вывод.

При подготовке данных не было обнаружено явных дубликатов, однако, во всех датасетах в столбце `id` присутствовали повторяющиеся значения. Предположив, что по этим скважинам были сделаны повторные замеры, было принято решение удалить повторы, оставив только последнее значение каждого из дубликатов.

## Обучение и проверка модели

In [24]:
def linear_model(data):
    target = data['product']
    features = data.drop(['id', 'product'], axis=1)
    
    features_train, features_valid, target_train, target_valid = train_test_split(features, target,
                                                                                  test_size=0.25, random_state=12345)
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    
    predicted_valid = model.predict(features_valid)
    
    rmse = mean_squared_error(target_valid, predicted_valid) ** 0.5
    
    # Средний запас предсказанного сырья
    product_mean = predicted_valid.sum() / len(predicted_valid)
    
    return product_mean, rmse, target_valid ,pd.Series(predicted_valid)

In [25]:
product_mean_1, rmse_1, target_valid_1, predicted_valid_1 = linear_model(region_1)
product_mean_2, rmse_2, target_valid_2, predicted_valid_2 = linear_model(region_2)
product_mean_3, rmse_3, target_valid_3, predicted_valid_3 = linear_model(region_3)

In [26]:
print('Регион 1. Средний запас сырья:', round(product_mean_1, 2), ', rmse:', round(rmse_1, 2))
print('Регион 2. Средний запас сырья:', round(product_mean_2, 2), ', rmse:', round(rmse_2, 2))
print('Регион 3. Средний запас сырья:', round(product_mean_3, 2), ', rmse:', round(rmse_3, 2))

Регион 1. Средний запас сырья: 92.6 , rmse: 37.93
Регион 2. Средний запас сырья: 68.53 , rmse: 0.89
Регион 3. Средний запас сырья: 94.94 , rmse: 40.19


### Вывод.

- Наибольшие средние запасы сырья наблюдаются в регионах 1 и 3, однако значения имеют большой разброс.
- В регионе 2 количество запасов меньше, но разброс минимальный.

## Подготовка к расчёту прибыли

In [27]:
REGION_BUDGET = 10 * 10**9
PRODUCT_INCOME = 450 * 10**3
TOTAL_POINTS = 500
SELECTED_POINTS = 200

Достаточный объем сырья для безубыточной разработки (тыс. баррелей).

In [28]:
min_volume_product = REGION_BUDGET / SELECTED_POINTS / PRODUCT_INCOME
min_volume_product

111.11111111111111

In [29]:
print('Регион 1. Отклонение от среднего:', round(product_mean_1 - min_volume_product, 2))
print('Регион 2. Отклонение от среднего:', round(product_mean_2 - min_volume_product, 2))
print('Регион 3. Отклонение от среднего:', round(product_mean_3 - min_volume_product, 2))

Регион 1. Отклонение от среднего: -18.52
Регион 2. Отклонение от среднего: -42.58
Регион 3. Отклонение от среднего: -16.18


### Вывод.

- Был рассчитан достаточный объем сырья для безубыточной разработки; он составил `111.111` единиц продукта.
- Во всех регионах наблюдается отклонение в меньшую сторону по средним запасам сырья.
- Среди всех регионов выделяется регион 2 - он имеет самое большое отклонение: `-42.6` единиц продукта.

## Расчёт прибыли и рисков 

In [30]:
# Расчет прибыли
def total_revenue(target, predicted):
    target = target.reset_index(drop=True)
    
    pred_sorted = predicted.sort_values(ascending=False)
    selected = target[pred_sorted.index][:SELECTED_POINTS]
    
    return selected.sum() * PRODUCT_INCOME - REGION_BUDGET

In [31]:
def bootstrap_revenue(target, predicted):
    state = np.random.RandomState(12345)
    
    bootstrap_samples = 1000
    values = []
    
    for i in range(bootstrap_samples):
        subsample = predicted.sample(n=TOTAL_POINTS, replace=True, random_state=state)
        
        values.append(total_revenue(target, subsample))
        
    values = pd.Series(values)
    
    # Средняя прибыль
    mean_revenue = values.mean()
    
    # 95% доверительный интервал
    lower = round(values.quantile(0.025), 2)
    upper = round(values.quantile(0.975), 2)
    
    # Вероятность убытка, %
    loss_risk = len(values[values < 0]) / len(values) * 100
    
    return round(mean_revenue, 2), lower, upper, round(loss_risk, 2)

In [32]:
mean_revenue_1, lower_1, upper_1, loss_risk_1 = bootstrap_revenue(target_valid_1, predicted_valid_1)
mean_revenue_2, lower_2, upper_2, loss_risk_2 = bootstrap_revenue(target_valid_2, predicted_valid_2)
mean_revenue_3, lower_3, upper_3, loss_risk_3 = bootstrap_revenue(target_valid_3, predicted_valid_3)

In [33]:
print('Регион 1. Средняя прибыль лучших месторождений:', f'{mean_revenue_1:_}')
print('Регион 2. Средняя прибыль лучших месторождений:', f'{mean_revenue_2:_}')
print('Регион 3. Средняя прибыль лучших месторождений:', f'{mean_revenue_3:_}')

Регион 1. Средняя прибыль лучших месторождений: 399_953_488.86
Регион 2. Средняя прибыль лучших месторождений: 441_524_243.74
Регион 3. Средняя прибыль лучших месторождений: 303_793_868.29


In [34]:
print('Регион 1. 95% доверительный интервал:', f'({lower_1:_}, {upper_1:_})')
print('Регион 2. 95% доверительный интервал:', f'({lower_2:_}, {upper_2:_})')
print('Регион 3. 95% доверительный интервал:', f'({lower_3:_}, {upper_3:_})')

Регион 1. 95% доверительный интервал: (-130_263_689.89, 916_732_138.96)
Регион 2. 95% доверительный интервал: (58_209_063.98, 838_801_420.05)
Регион 3. 95% доверительный интервал: (-226_395_997.83, 861_077_922.29)


In [35]:
print('Регион 1. Риск убытков, %:', loss_risk_1)
print('Регион 2. Риск убытков, %:', loss_risk_2)
print('Регион 3. Риск убытков, %:', loss_risk_3)

Регион 1. Риск убытков, %: 6.6
Регион 2. Риск убытков, %: 1.4
Регион 3. Риск убытков, %: 12.3


## Вывод.

- Было проведено исследование, целью которого было выяснить, какой из регионов принесет наибольшую прибыль при его разработке. Для этого было проделано следующее:
    - Загружены и обработаны данные;
    - Произведен расчет достаточного объема сырья для безубыточной разработки;
    - Обучена модель линейной регрессии;
    - Проведен расчет прибыли на основании 200 лучших месторождений в каждом регионе;
    - Определены 95% доверительные интервалы;
    - Произведен расчет риска убытков в каждом регионе.
- Получили следующие данные:
    - Достаточный объем сырья составил `111.111` единиц продукта.
    - Во всех регионах средние запасы продукта оказались ниже достаточного объема:
        - Регион 1. Отклонение от среднего: `-18.52`
        - Регион 2. Отклонение от среднего: `-42.58`
        - Регион 3. Отклонение от среднего: `-16.18`
    - Применили технику `bootstrap` на 1000 выборок для распределения прибыли и вычислили прибыль для `200` лучших месторождений в каждом регионе:
        - Регион 1. Средняя прибыль лучших месторождений: `399_953_488.86`
        - Регион 2. Средняя прибыль лучших месторождений: `441_524_243.74`
        - Регион 3. Средняя прибыль лучших месторождений: `303_793_868.29`
    - 95% доверительные интервалы в каждом регионе:
        - Регион 1. 95% доверительный интервал: `(-130_263_689.89, 916_732_138.96)`
        - Регион 2. 95% доверительный интервал: `(58_209_063.98, 838_801_420.05)`
        - Регион 3. 95% доверительный интервал: `(-226_395_997.83, 861_077_922.29)`
    - Значение рисков для каждого региона:
        - Регион 1. Риск убытков, %: `6.6`
        - Регион 2. Риск убытков, %: `1.4`
        - Регион 3. Риск убытков, %: `12.3`
- На основании полученных данных рекомендуемым регионом для разработки является `регион 2`. Несмотря на то, что средние запасы сырья имеют наибольшее отклонение от достаточного объема, в этом регионе наибольшая прибыль для 200 лучших месторождений, самые узкие рамки доверительного интервала, самые низкие риски понести убытки.